<a href="https://colab.research.google.com/github/obijywk/grilops/blob/master/examples/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# grilops tutorial

This notebook will step through how to solve some logic puzzles using grilops and z3.

## Setup

First, we'll need to make sure the `grilops` package is installed. This will also install the `z3-solver` package if needed (as it is a dependency of `grilops`).

In [1]:
import sys
!{sys.executable} -m pip install grilops

Next, we'll import the `grilops` module, and everything from the `z3` module (some consider wildcard imports to be an [anti-pattern](https://docs.quantifiedcode.com/python-anti-patterns/maintainability/from_module_import_all_used.html) in Python, but doing this is convenient for the purposes of this tutorial).

In [0]:
import grilops
from z3 import *

Now we can move on to solving some puzzles!

## Sudoku

[Sudoku](https://en.wikipedia.org/wiki/Sudoku) is a good puzzle to start with, because it's well-known, and is relatively simple to model.

We'll start by creating a list of lists containing the pre-filled numbers given in the puzzle. We'll use the givens from the example puzzle from Wikipedia. We'll use a 0 to represent a cell for which we don't have a given value.

In [0]:
  givens = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
  ]

Now let's create a grilops `SymbolSet` to model the marks that we can fill into the grid (in this case, the digits 1 through 9), and a 9x9 grilops `SymbolGrid` to model the grid itself. See the grilops [Symbols](https://obijywk.github.io/grilops/symbols/) and [Grids](https://obijywk.github.io/grilops/grids/) documentation to learn more about these objects.

In [0]:
sym = grilops.make_number_range_symbol_set(1, 9)
sg = grilops.SymbolGrid(9, 9, sym)

Our next step will be to enter our given numbers into the grid. We'll do this by looping over all of the positions in the grid, and constraining the grid to contain the given number at that position whenever it is not 0.

In [0]:
for y in range(9):
  for x in range(9):
    given = givens[y][x]
    if given != 0:
      sg.solver.add(sg.cell_is(y, x, sym[given]))

When the `SymbolGrid` was constructed, it created a z3 [`Solver`](https://z3prover.github.io/api/html/classz3py_1_1_solver.html) object, accessible via its `solver` attribute. We'll use this solver to add all of our puzzle-specific constraints, and ultimately to solve the puzzle.

The `SymbolGrid.cell_is` method returns a constraint requiring that a cell at a given position in the grid contains a given symbol. Notice that the y (vertical) coordinate comes before the x (horizontal) coordinate; this matches the order we used to define our grid of givens, and is a convention used throughout grilops. Also note that we're looking up the symbol for the given digit, rather than passing in the given digit directly; this is important (and a little unintuitive) in this case, because our symbol set containing the digits 1 through 9 is actually indexed starting from 0!

Next, let's add the defining constraints of Sudoku: each row, column, and 3x3 subgrid may only contain each digit one time. We'll use the z3 `Distinct` operator to express this.

In [0]:
rows = sg.grid
for row in rows:
  sg.solver.add(Distinct(*row))

columns = [[sg.grid[y][x] for y in range(9)] for x in range(9)]
for column in columns:
  sg.solver.add(Distinct(*column))

for subgrid_index in range(9):
  top = (subgrid_index // 3) * 3
  left = (subgrid_index % 3) * 3
  cells = [sg.grid[y][x] for y in range(top, top + 3) for x in range(left, left + 3)]
  sg.solver.add(Distinct(*cells))

Okay, we've added all of the constraints needed to model a Sudoku puzzle. Now let's try to solve it!

In [7]:
sg.solve()

True

`True` means we found a solution! Let's see what it is.

In [8]:
sg.print()

534678912
672195348
198342567
859761423
426853791
713924856
961537284
287419635
345286179


Looks good!

Let's check to see if there are any other possible solutions to this puzzle.

In [9]:
sg.is_unique()

True

This solution is unique. If it had turned out not to be unique (if there were an alternate solution) we could now call `sg.print()` again to see the alternate solution.

## Fillomino

Let's try a [Fillomino](https://en.wikipedia.org/wiki/Fillomino) puzzle now. This example will demonstrate the use of the grilops [`RegionConstrainer`](https://obijywk.github.io/grilops/regions/) to divide the grid into orthogonally contiguous regions of cells (polyominoes).

We'll start by creating a list of lists of the given region sizes, using 0 to indicate a cell that does not contain a given value.

In [0]:
givens = [
  [0, 0, 0, 3, 0, 0, 0, 0, 5],
  [0, 0, 8, 3, 10, 0, 0, 5, 0],
  [0, 3, 0, 0, 0, 4, 4, 0, 0],
  [1, 3, 0, 3, 0, 0, 2, 0, 0],
  [0, 2, 0, 0, 3, 0, 0, 2, 0],
  [0, 0, 2, 0, 0, 3, 0, 1, 3],
  [0, 0, 4, 4, 0, 0, 0, 3, 0],
  [0, 4, 0, 0, 4, 3, 3, 0, 0],
  [6, 0, 0, 0, 0, 1, 0, 0, 0],
]

Now we'll create our SymbolSet and our SymbolGrid.

In [0]:
sym = grilops.make_number_range_symbol_set(1, 10)
sg = grilops.SymbolGrid(9, 9, sym)

Note that we're assuming that there will not be any region larger than 10 cells (the upper bound of our number range symbol set). We could make this upper bound arbitrarily large, but doing so might increase the search space, causing the solver to take longer to run.

Now we'll introduce a `RegionConstrainer` set up to use the same solver as our `SymbolGrid`. We'll need to import the `grilops.regions` module to use this class.

In [0]:
import grilops.regions
rc = grilops.regions.RegionConstrainer(9, 9, solver=sg.solver)

Okay, now we can start adding the constraints that define the logic of the puzzle.

First, we'll associate each symbol in the grid with the concept that its value represents: the size of the region to which the cell belongs. The `RegionConstrainer` provides us with a `region_size_grid` where each cell contains the size of that cell's region.

In [0]:
for y in range(9):
  for x in range(9):
    # Note that the symbol for 1 actually has index 0, so we must subtract
    # 1 from the size of the region when adding this constraint.
    sg.solver.add(sg.grid[y][x] == rc.region_size_grid[y][x] - 1)

Next, we'll add a constraint for each of our givens ensuring that the size of the region matches the given's value.

In [0]:
for y in range(9):
  for x in range(9):
    given = givens[y][x]
    if given != 0:
      sg.solver.add(rc.region_size_grid[y][x] == given)

Finally, Fillomino requires that "no two polyominoes of matching size (number of cells) are orthogonally adjacent (share a side)." To add this constraint, we'll consider the orthogonal neighbors of each cell, and ensure that if two orthogonally adjacent cells have the same region size, that they are also part of the same region. We'll implement the "part of the same region" constraint using the `region_id_grid` attribute of the `RegionConstrainer`; in this grid, each cell will contain a numeric identifier that is shared among all cells that are part of the same region.

In [0]:
for y in range(9):
  for x in range(9):
    adjacent_sizes = grilops.adjacent_cells(rc.region_size_grid, y, x)
    adjacent_ids = grilops.adjacent_cells(rc.region_id_grid, y, x)
    for adjacent_size, adjacent_id in zip(adjacent_sizes, adjacent_ids):
      sg.solver.add(
          Implies(
              rc.region_size_grid[y][x] == adjacent_size,
              rc.region_id_grid[y][x] == adjacent_id
          )
      )

And that's it! Time to solve.

In [16]:
sg.solve()

True

In [17]:
sg.print()

8 8 3 3 101010105 
8 8 8 3 1010105 5 
3 3 8 10104 4 4 5 
1 3 8 3 102 2 4 5 
2 2 8 3 3 1 3 2 2 
6 6 2 2 1 3 3 1 3 
6 4 4 4 2 2 1 3 3 
6 4 2 2 4 3 3 4 4 
6 6 4 4 4 1 3 4 4 


In [18]:
sg.is_unique()

True